In [39]:
import tarfile
from zlib import decompress
import functools
import zstandard

In [1]:
import os
from pathlib import Path

In [1]:
from datetime import datetime
import polars as pl

In [5]:
metadata = pl.read_csv('raw/metadata.csv', try_parse_dates=True)
metadata

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""0003de2700e20a1681d69fe287441d…",2017-05-31 01:00:00 UTC,2019-11-14 00:00:00 UTC,897.0,2.455852,21528,21476,52,0.241546,2019-09-02,2019-11-13,"""2.0A""","""""",4.6,null,null,null,null,null,"""Gipuzkoa""","""Donostia/San Sebastian""",20013,9820
"""0004150214d14a2b2e6f7075531e66…",2018-07-12 01:00:00 UTC,2022-06-05 00:00:00 UTC,1424.0,3.8987,34176,34169,7,0.020482,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Irun""",20304,4759
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ffeddf4919e748beb50af7c6b73def…",2018-10-02 01:00:00 UTC,2021-11-03 00:00:00 UTC,1128.0,3.088296,27072,27069,3,0.011082,2021-06-01,null,"""2.0TD""","""""",6.0,6.0,null,null,null,null,"""Soria""",null,null,9820
"""ffefb9e27715b1c3fa5007c457de55…",2021-07-06 01:00:00 UTC,2022-06-05 00:00:00 UTC,334.0,0.914442,8016,8014,2,0.02495,2021-07-07,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48012,9820
"""fff0adcd6cdb7434496a5222f03337…",2017-08-17 01:00:00 UTC,2022-06-05 00:00:00 UTC,1753.0,4.799452,42072,41102,970,2.305571,2021-06-01,null,"""2.0TD""","""""",4.6,4.6,null,null,null,null,"""Bizkaia""",null,null,9820


In [6]:
# metadata['start_date'] = pl.date(metadata['start_date'])
# metadata['end_date'] = pl.datetime(metadata['end_date'])

In [10]:
# metadata_post_covid = metadata[metadata['start_date'] > pd.to_datetime('2021-05-31T00:00:00Z')]
metadata_post_covid = metadata.filter(
    pl.col("start_date").is_between(pl.datetime(2021, 5, 31, time_zone="UTC"), pl.datetime(3000, 1, 1, time_zone="UTC"))
)
metadata_post_covid.head()

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""001969772145e8c6bb738528594aef…",2021-12-14 01:00:00 UTC,2022-06-05 00:00:00 UTC,173.0,0.473648,4152,4151,1,0.024085,2021-12-15,null,"""2.0TD""","""""",4.4,4.4,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48013,null
"""001e11d0fbe42f2b119741d5af6f4a…",2021-07-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,309.0,0.845996,7416,7414,2,0.026969,2021-07-31,null,"""2.0TD""","""""",5.75,5.75,0.0,0.0,0.0,0.0,"""Araba/Alava""","""Vitoria-Gasteiz""",1015,9820
"""0024e07345b50dd266c6390d5c496a…",2021-06-03 01:00:00 UTC,2022-06-05 00:00:00 UTC,367.0,1.004791,8808,8806,2,0.022707,2021-06-04,null,"""2.0TD""","""""",4.4,4.4,0.0,0.0,0.0,0.0,"""Navarra""",null,null,9820
"""00286b6d86e8094a80c6efeec03c55…",2021-09-01 01:00:00 UTC,2022-06-05 00:00:00 UTC,277.0,0.758385,6648,6646,2,0.030084,2021-09-02,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9820


In [11]:
metadata_post_covid['municipality'].value_counts()

municipality,count
str,u32
"""Donostia/San Sebastian""",282
"""Majadahonda""",1
"""Sagunto/Sagunt""",1
"""Barakaldo""",23
"""Malaga""",5
…,…
"""Caceres""",8
"""Valdemoro""",2
"""Madrid""",178


In [20]:
metadata_post_covid.filter(metadata_post_covid['province'].is_not_null())

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""001969772145e8c6bb738528594aef…",2021-12-14 01:00:00 UTC,2022-06-05 00:00:00 UTC,173.0,0.473648,4152,4151,1,0.024085,2021-12-15,null,"""2.0TD""","""""",4.4,4.4,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48013,null
"""001e11d0fbe42f2b119741d5af6f4a…",2021-07-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,309.0,0.845996,7416,7414,2,0.026969,2021-07-31,null,"""2.0TD""","""""",5.75,5.75,0.0,0.0,0.0,0.0,"""Araba/Alava""","""Vitoria-Gasteiz""",1015,9820
"""0024e07345b50dd266c6390d5c496a…",2021-06-03 01:00:00 UTC,2022-06-05 00:00:00 UTC,367.0,1.004791,8808,8806,2,0.022707,2021-06-04,null,"""2.0TD""","""""",4.4,4.4,0.0,0.0,0.0,0.0,"""Navarra""",null,null,9820
"""00286b6d86e8094a80c6efeec03c55…",2021-09-01 01:00:00 UTC,2022-06-05 00:00:00 UTC,277.0,0.758385,6648,6646,2,0.030084,2021-09-02,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9820
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ffe0fec04b184324aaef086e779326…",2021-12-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,156.0,0.427105,3744,3743,1,0.026709,2022-01-01,null,"""2.0TD""","""""",3.3,3.3,null,null,null,null,"""Gipuzkoa""",null,null,9820
"""ffe139ac875ad687fa4f02fad29f12…",2021-09-17 01:00:00 UTC,2022-03-05 00:00:00 UTC,169.0,0.462697,4056,4055,1,0.024655,2021-09-18,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Araba/Alava""",null,null,0
"""ffefb9e27715b1c3fa5007c457de55…",2021-07-06 01:00:00 UTC,2022-06-05 00:00:00 UTC,334.0,0.914442,8016,8014,2,0.02495,2021-07-07,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48012,9820


In [30]:
def getUniqueCount(df:pl.DataFrame, group_by:str):
    return df.group_by(group_by).len().sort('len', descending=True)

In [33]:
getUniqueCount(metadata_post_covid, 'province')

province,len
str,u32
"""Gipuzkoa""",1491
"""Bizkaia""",913
"""Navarra""",640
"""Araba/Alava""",414
"""Madrid""",252
…,…
"""Avila""",1
"""Valladolid""",1
"""Pontevedra""",1


In [14]:
import glob
import pandas as pd

def combine_csv_files(source_folder, output_file):
    # Get a list of all .csv files in the source folder
    csv_files = glob.glob(f"{source_folder}/*.csv")

    # Create an empty list to store dataframes
    dataframes = []

    # Loop through each .csv file, read it, and append it to the list of dataframes
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        dataframes.append(df)

    # Concatenate all dataframes into one
    combined_df = pd.concat(dataframes, ignore_index=True)

    # Save the combined dataframe to a new .csv file
    combined_df.to_csv(output_file, index=False)
    print(f"Combined CSV saved to {output_file}")


In [15]:
combine_csv_files("dataset/goi4_pst/imp_csv", "output.csv")

Combined CSV saved to output


In [17]:
import csv

def combine_csv_files(source_folder, output_file):
    # Get a list of all .csv files in the source folder
    csv_files = glob.glob(f"{source_folder}/*.csv")

    # Open the output file in write mode
    with open(output_file, mode='w', newline='') as outfile:
        writer = csv.writer(outfile)

        # Variable to track if headers have been written
        headers_written = False

        # Loop through each .csv file
        for file_path in csv_files:

            # Open each CSV file in read mode
            with open(file_path, mode='r', newline='') as infile:
                reader = csv.reader(infile)

                # Write headers only once
                if not headers_written:
                    headers = next(reader)  # Read the header of the first file
                    writer.writerow(headers)  # Write the header to the output file
                    headers_written = True
                else:
                    next(reader)  # Skip the header of subsequent files

                # Write the data rows to the output file
                for row in reader:
                    writer.writerow(row)


combine_csv_files("dataset/goi4_pst/imp_csv", "output.csv")